# CIFAR10 인식 정확도 챌린지 

### 컴퓨터과학과 201910976 안선정

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.optim import lr_scheduler
scaler = StandardScaler()
batch_size = 32
learning_rate = 0.01
num_epoch = 37

In [ ]:
cifar_train=dset.CIFAR10("CIFAR10/", train=True, 
                        transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(0.5,0.5,0.5), std = (0.5,0.5,0.5)),
                        ])
                        , target_transform= None, download=True)
cifar_test=dset.CIFAR10("CIFAR10/", train = False, 
                        transform = transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize(mean=(0.5,0.5,0.5), std = (0.5,0.5,0.5)),
                        ])
                        , target_transform= None, download=False)


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/


In [ ]:
def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0
    
    for j, [imgs, labels] in enumerate(dloader):
        img = Variable(imgs, volatile=True).cuda()
        label = Variable(labels).cuda()
        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)
        total += label.size(0)
        correct += (output_index == label).sum().float()
    print("Accuracy of Test Data:{}".format(100*correct/total))

## 모델 수정

In [ ]:
train_loader = torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers= 2, drop_last = True)
test_loader = torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers = 2, drop_last = True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3,16,3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,32,3,padding = 1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Dropout2d(p=0.07),
            nn.Conv2d(32, 64, 3, padding = 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Dropout2d(p=0.1),
            nn.Linear(64*8*8, 100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout2d(p=0.1),
            nn.Linear(100,10)
        )
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
            if isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)

    def forward(self, x):
        out=self.layer(x)
        out=out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

model=CNN().cuda()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.


## 모델 학습

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.train()
for i in range(num_epoch):
    for j,[image, label] in enumerate(train_loader):
        x=Variable(image).cuda()
        y_=Variable(label).cuda()

        optimizer.zero_grad()
        output=model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()

        if j%1000 ==0:
            print(j, loss)

0 tensor(0.6589, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.5631, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.2921, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.5442, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.2600, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.7729, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.1411, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.3345, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.2832, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.2805, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.5250, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.1256, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.1871, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.2151, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.2979, device='cuda:0', grad_fn=<NllLossBackward>)
1000 tensor(0.2916, device='cuda:0', grad_fn=<NllLossBackward>)


## 모델 저장 전 test_loader의 정확도

In [ ]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data:78.0849380493164


## 모델 저장

In [ ]:
path = './content'

In [ ]:
torch.save(model, path+'78.08.pkl')

## 저장된 모델 load후 test_loader의 정확도

In [ ]:
netname = '/content/content78.08.pkl'
model = torch.load(netname)
model.eval()
ComputeAccr(test_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data:78.0849380493164
